In [43]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import illustrisAPI.data as iApi
%matplotlib inline

In [44]:
def getData(snap):
    fileName='Data/full_data/FullData_Snap.'+str(snap)+'.txt'
    fieldwidth = 16
    numfields = 5
    #data with gaps
    borkedData=np.genfromtxt(fileName,delimiter=(fieldwidth,)*numfields)
    #each part of the rows of data
    a=borkedData[::4,:]
    b=borkedData[1::4,:]
    c=borkedData[2::4,:]
    d=borkedData[3::4,:3]
    nGal=np.shape(a)[0]
    #extra space for new data we're adding
    e=np.zeros((nGal,6))
    data=np.hstack([a,b,c,d,e])
    return data
def getHistory(sub):
    passFail=1
    fileName='Data/AccHistory/AccHistInt_SubGr.'+str(int(sub))+'.txt'
    fieldwidth = 16
    numfields = 5
    #data with gaps
    borkedHistory=np.genfromtxt(fileName,delimiter=(fieldwidth,)*numfields)
    a=borkedHistory[::4,:]
    b=borkedHistory[1::4,:]
    c=borkedHistory[2::4,:]
    d=borkedHistory[3::4,:1]
    if (d.shape[0]==0):
        #print('sub: ',sub,' is weird, skipping...')
        return np.zeros(1)
    history=np.hstack([a,b,c,d])
    return history

In [46]:
nSkips=0
nSkipsAlt=0
z0data=getData(135)
z0subs=z0data[:,1]
for snap in range(130,136):
    print('starting snap: ',snap)
    zData=getData(snap)
    
    mBh=iApi.getSubhaloField('SubhaloBHMass',snapshot=snap,simulation='Illustris-1')
    mDotBh=iApi.getSubhaloField('SubhaloBHMdot',snapshot=snap,simulation='Illustris-1')
    sfr=iApi.getSubhaloField('SubhaloSFR',snapshot=snap,simulation='Illustris-1')
    
    zData[:,18]=mBh[zData[:,1].astype(int)]*0.719 #sim units as 10^10 M_odot /h -> 10^10 M_odot
    zData[:,19]=mDotBh[zData[:,1].astype(int)]*0.978#sim units as 10^10 M_odot /h (0.978 Gyr/h) -> 10^10 M_odot / Gyr
    zData[:,20]=sfr[zData[:,1].astype(int)]/10#sim units as M_odot/yr -> 10^10 M_odot / Gyr
    zData[:,21]=-1*np.ones(np.shape(zData)[0]) #initializing these values as -1
    zData[:,22]=-1*np.ones(np.shape(zData)[0])
    zData[:,23]=-1*np.ones(np.shape(zData)[0])
    
    zSubs=zData[:,1]
    history=getHistory(0)
    for sub in z0subs:
        history=getHistory(sub)
        if (history.size==1):
            continue
        thisEntry=np.argwhere((history[:,0]==0) & (history[:,1]==snap))
        if (thisEntry.size==0):
            nSkips+=1
            continue
        thisSub=int(history[thisEntry,4])
        zEntry=np.argwhere(zSubs==thisSub)
        zData[zEntry,21]=sub
        if (snap<135):
            nextEntry=np.argwhere((history[:,0]==0) & (history[:,1]==snap+1))
            if (nextEntry.size==0):
                nSkipsAlt+=1
                continue
            nextSub=history[nextEntry,4]
            zData[zEntry,22]=nextSub
        if (snap>85):
            prevEntry=np.argwhere((history[:,0]==0) & (history[:,1]==snap-1))
            if (prevEntry.size==0):
                nSkipsAlt+=1
                continue
            prevSub=history[prevEntry,4]
            zData[zEntry,23]=prevSub
    saveName='Data/GalaxyData/GalaxyData_Snap.'+str(snap)+'.txt'
    np.savetxt(saveName,zData)

starting snap:  130


OSError: Unable to open file (Truncated file: eof = 2223806, sblock->base_addr = 0, stored_eoa = 17633068)

In [18]:
print(nSkips)
print(nSkipsAlt)

6
12
